#**DATA EXPLORATION**

**SUMMARY**
1. Check for missing values and empty strings
2. Looking at the text in depth to see what expressions can be removed
3. Looking at the each catergy size


In [1]:
import pandas as pd
df=pd.read_json("/problems_data.jsonl",lines=True)

In [2]:
df.shape

(4112, 8)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4112 entries, 0 to 4111
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               4112 non-null   object 
 1   description         4112 non-null   object 
 2   input_description   4112 non-null   object 
 3   output_description  4112 non-null   object 
 4   sample_io           4112 non-null   object 
 5   problem_class       4112 non-null   object 
 6   problem_score       4112 non-null   float64
 7   url                 4112 non-null   object 
dtypes: float64(1), object(7)
memory usage: 257.1+ KB


In [4]:
df.problem_class.value_counts()

,count
problem_class,
hard,1941
medium,1405
easy,766


In [29]:
category_stats = df.groupby('problem_class')['problem_score'].agg(['count', 'mean', 'std', 'min', 'max']).round(2)
print("Category Statistics:")
print(category_stats)

Category Statistics:
               count  mean   std  min  max
problem_class                             
easy             766  1.97  0.43  1.1  2.8
hard            1941  7.07  1.05  5.5  9.7
medium          1405  4.13  0.77  2.8  5.5


In [5]:
#the above tells us there is imabalance, we shall have to balance it.

In [6]:
df.head()

,title,description,input_description,output_description,sample_io,problem_class,problem_score,url
0,Uuu,Unununium (Uuu) was the name of the chemical\n...,The input consists of one line with two intege...,The output consists of $M$ lines where the $i$...,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 ...",hard,9.7,https://open.kattis.com/problems/uuu
1,House Building,A number of eccentrics from central New York h...,"The input consists of $10$ test cases, which a...",Print $K$ lines with\n the positions of the...,"[{'input': '0 2 3 2 50 60 50 30 50 40', 'outpu...",hard,9.7,https://open.kattis.com/problems/husbygge
2,Mario or Luigi,Mario and Luigi are playing a game where they ...,,,"[{'input': '', 'output': ''}]",hard,9.6,https://open.kattis.com/problems/marioorluigi
3,The Wire Ghost,Žofka is bending a copper wire. She starts wit...,The first line contains two integers $L$ and $...,The output consists of a single line consistin...,"[{'input': '4 3 3 C 2 C 1 C', 'output': 'GHOST...",hard,9.6,https://open.kattis.com/problems/thewireghost
4,Barking Up The Wrong Tree,"Your dog Spot is let loose in the park. Well, ...",The first line of input consists of two intege...,Write a single line containing the length need...,"[{'input': '2 0 10 0 10 10', 'output': '14.14'...",hard,9.6,https://open.kattis.com/problems/barktree


In [7]:
#the title,sample_io,url does not seem to contribute to guess the difficulty level so they should be removed

In [14]:
#checking for null values
df.isna().sum()

,0
title,0
description,0
input_description,0
output_description,0
sample_io,0
problem_class,0
problem_score,0
url,0


In [16]:
#checking for empty strings
(df['description'] == "").sum()

np.int64(81)

In [20]:
#checking for white space
df['description'].str.strip().eq("").sum()

np.int64(81)

In [17]:
#checking for fake null
df['description'].str.lower().isin(['nan']).sum()

np.int64(0)

In [26]:
pd.DataFrame({
    'nulls': df[['title','description','input_description','output_description','sample_io']].isna().sum(),
    'empty_strings': (df[['title','description','input_description','output_description','sample_io']] == "").sum(),
    'fake_null': (df[['title','description','input_description','output_description','sample_io']].isin(['none','nan','null'])).sum()
})


,nulls,empty_strings,fake_null
title,0,0,0
description,0,81,0
input_description,0,120,0
output_description,0,131,0
sample_io,0,0,0


In [27]:
#we have to handle the empty strings

In [8]:
pd.set_option("display.max_colwidth", None)

In [9]:
df.loc[0]

,0
title,Uuu
description,"Unununium (Uuu) was the name of the chemical\n element with atom number 111, until it changed to\n Röntgenium (Rg) in 2004. These heavy elements are very\n unstable and have only been synthesized in a few\n laboratories.\nYou have just been hired by one of these labs to optimize\n the algorithms used in simulations. For example, when\n simulating complicated chemical reactions, it is important to\n keep track of how many particles there are, and this is done by\n counting connected components in a graph.\nCurrently, the lab has some Python code (see attachments)\n that takes an undirected graph and outputs the number of\n connected components. As you can see, this code is based on\n everyone’s favourite data structure union-find1.\nAfter looking at the code for a while, you notice that it\n actually has a bug in it! The code still gives correct answers,\n but the bug could cause it to run inefficiently. Your task is\n to construct a graph with a given number of vertices and edges\n where the code runs very slowly. We will count how many times\n the third line (the one inside the while loop) is visited, and\n your program will get a score according to this number.\n"
input_description,"The input consists of one line with two integers\n $N$ and $M$, the number of vertices and edges\n your graph should have. Apart from the sample, there will be\n only one test case, with $N =\n 100$ and $M =\n 500$."
output_description,"The output consists of $M$ lines where the $i$:th contains two integers\n $u_ i$ and $v_ i$ ($1 \leq u_ i, v_ i \leq N$). This\n indicates that the vertices $u_\n i$ and $v_ i$ are\n connected with an edge in your graph."
sample_io,"[{'input': '7 10', 'output': '1 2 2 3 1 3 3 4 5 6 6 7 5 7 1 7 7 2 5 1'}]"
problem_class,hard
problem_score,9.7
url,https://open.kattis.com/problems/uuu


In [10]:
'''
1. we should keep the numbers and math symbols for sure as they tell how tough a problem could be.
2.we can remove () \n ! , ' -
3. we can also covert the special characters like ö to normal characters.
4. we can lowercase everything.
'''

'\n1. we should keep the numbers and math symbols for sure as they tell how tough a problem could be.\n2.we can remove () \n ! , \n3. we can also covert the special characters like ö to normal characters.\n4. we can lowercase everything.\n'

In [11]:
#checking any random row for more ideas to clean
row = df.sample(1).index[0]
df.loc[row]


,2449
title,Eko
description,"Lumberjack Mirko needs to chop down $M$ metres of wood. It is an easy job\n for him since he has a nifty new woodcutting machine that can\n take down forests like wildfire. However, Mirko is only allowed\n to cut a single row of trees.\nMirko’s machine works as follows: Mirko sets a height\n parameter $H$ (in metres),\n and the machine raises a giant sawblade to that height and cuts\n off all tree parts higher than $H$ (of course, trees not higher than\n $H$ meters remain intact).\n Mirko then takes the parts that were cut off. For example, if\n the tree row contains trees with heights of $20$, $15$, $10$, and $17$ metres, and Mirko raises his\n sawblade to $15$ metres,\n the remaining tree heights after cutting will be $15$, $15$, $10$, and $15$ metres, respectively, while Mirko\n will take $5$ metres off\n the first tree and $2$\n metres off the fourth tree ($7$ metres of wood in total).\nMirko is ecologically minded, so he does not want to cut off\n more wood than necessary. That is why he wants to set his\n sawblade as high as possible. Help Mirko find the maximum\n integer height of the sawblade that still allows him to cut off\n at least $M$ metres of\n wood.\n"
input_description,"The first line of input contains two space-separated\n positive integers, $N$\n (the number of trees, $1 \le N\n \le 1\, 000\, 000$) and $M$ (Mirko’s required wood amount,\n $1 \le M \le 2\, 000\, 000\,\n 000$)."
output_description,The first and only line of output must contain the required\n height setting.
sample_io,"[{'input': '4 7 20 15 10 17', 'output': '15'}, {'input': '5 20 4 42 40 26 46', 'output': '36'}]"
problem_class,medium
problem_score,4.5
url,https://open.kattis.com/problems/eko


In [13]:
#1. we can remove \cdot _ :

So in conclusion:
1. there is imabalance, we shall have to balance it.
2. the title,sample_io,url does not seem to contribute to guess the difficulty level so they should be removed
3. we have to handle the empty strings
4. we should keep the numbers and math symbols for sure as they tell how tough a problem could be.
5. we can remove () \n ! , \cdot _ : ' -
6. we can also covert the special characters like ö to normal characters.
7. we can lowercase everything.


